# EDA

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
from datetime import datetime
from numpy.linalg import svd
from sklearn.decomposition import TruncatedSVD

import csv
import os
import sys


In [2]:
DATA_PATH = '/opt/ml/input/data/train_dataset/'
train_df = pd.read_csv(DATA_PATH+'cv_train_data_FE.csv')
val_df = pd.read_csv(DATA_PATH+'cv_valid_data_FE.csv')
# test_fe_df = pd.read_pickle(DATA_PATH+'train_data_FE_0609.pkl')

In [3]:
train_df.head(20)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,hour,dow,elapsed,grade,...,retest,solved_disorder,last_problem,answer_delta,tag_delta,test_delta,assess_delta,left_asymptote,elo_prob,cum_correct
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,1,0.0,6,...,0,0,0,0.290768,0.044978,0.052317,0.017937,0,0.979350,1.000000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0,1,3.0,6,...,0,0,0,0.290768,0.086813,0.052317,0.035874,0,0.970579,1.000000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0,1,8.0,6,...,0,0,0,0.290768,0.086813,0.052317,0.089686,0,0.942168,1.000000
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0,1,7.0,6,...,0,0,0,0.290768,0.086813,0.052317,0.031390,0,0.972448,1.000000
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0,1,7.0,6,...,0,0,0,0.290768,0.086813,0.052317,0.058296,0,0.957230,1.000000
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,0,1,11.0,6,...,0,1,-1,0.290768,0.086813,0.052317,0.080717,0,0.947445,1.000000
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,5,3,NaN,6,...,0,0,0,-0.709232,-0.799552,-0.790562,-0.882353,0,0.912392,0.000000
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,5,3,7.0,6,...,0,0,0,0.290768,0.200448,0.209438,0.081448,0,0.937566,0.000000
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,5,3,64.0,6,...,0,0,0,0.290768,0.200448,0.209438,0.479638,0,0.628990,0.500000
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,5,3,15.0,6,...,0,0,0,0.290768,0.200448,0.209438,0.176471,0,0.871671,0.666667


In [4]:
train_df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'hour', 'dow', 'elapsed', 'grade', 'mid',
       'problem_number', 'test_mean', 'test_sum', 'tag_mean', 'tag_sum',
       'ass_mean', 'ass_sum', 'prb_mean', 'prb_sum', 'hour_mean', 'hour_sum',
       'dow_mean', 'dow_sum', 'tag_elp', 'tag_elp_o', 'tag_elp_x', 'ass_elp',
       'ass_elp_o', 'ass_elp_x', 'prb_elp', 'prb_elp_o', 'prb_elp_x',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'Grade_o',
       'GradeCount', 'GradeAcc', 'GradeElp', 'GradeMElp', 'problem_count',
       'tag_count', 'RepeatedTime', 'prior_KnowledgeTag_frequency',
       'problem_position', 'solve_order', 'retest', 'solved_disorder',
       'last_problem', 'answer_delta', 'tag_delta', 'test_delta',
       'assess_delta', 'left_asymptote', 'elo_prob', 'cum_correct'],
      dtype='object')

In [23]:
test_fe_df= pd.read_csv(DATA_PATH+'train_FE.csv')

In [ ]:
test_fe_df['Timestamp'] = test_fe_df['Timestamp'].apply(convert_time)
test_fe_df.to_csv(os.path.join("/opt/ml/input/data/train_dataset","train_FE.csv"), index=False)

In [ ]:
# test_fe_df['Timestamp'].unique()
test_fe_df.to_csv(os.path.join("/opt/ml/input/data/train_dataset","train_FE.csv"), index=False)

In [ ]:
test_fe_df.to_csv(os.path.join("/opt/ml/input/data/train_dataset","TrainFE_lf.csv"), index=False)

In [ ]:
test_df.to_csv(os.path.join("/opt/ml/input/data/train_dataset","test_val.csv"), index=False)

In [ ]:
DATA_PATH = '/opt/ml/input/data/train_dataset/'
train_df = pd.read_csv(DATA_PATH+'cv_train_data.csv', parse_dates=['Timestamp'], index_col=0)
# train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
val_df = pd.read_csv(DATA_PATH+'cv_valid_data.csv', parse_dates=['Timestamp'], index_col=0)
# test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [ ]:
test_df = pd.read_csv('/opt/ml/input/data/train_dataset/train_data.csv')

In [ ]:
test_df

In [ ]:
test_df['Timestamp'] = test_df['Timestamp'].apply(convert_time)

In [ ]:
test_df.to_csv(os.path.join("/opt/ml/input/data/train_dataset","train_cached.csv"), index=False)

In [ ]:

import time
from datetime import datetime
def convert_time(s):
    timestamp = time.mktime(datetime.strptime(str(s), '%Y-%m-%d %H:%M:%S').timetuple())
    return int(timestamp)

In [ ]:
import os
import pandas as pd
import json
from tqdm.auto import tqdm
def feature_creator(exp_dir:str, col_name:str, data_dict:dict):
    for mode, csv_file in tqdm(data_dict.items(), desc="processing features..."):
        df = pd.read_csv(csv_file)
        df['temp_idx'] = df.index
    
        if df['Timestamp'].dtype == object or df['Timestamp'].dtype == "datetime64[ns]":
            df['Timestamp'] = df['Timestamp'].apply(convert_time)
        #### YOUR CODE HERE FROM HERE ####
        # if mode == "test": 
            # test의 경우 다른 전처리가 필요한 경우의 분기 코드
            # (ex) test dataset의 경우, answerCode가 일부는 -1로 마스킹되어 있음. 이를 처리하기 위한 다른 코드 필요)
        # else:
        df = df.sort_values(by=['userID', 'Timestamp'], axis=0)
        df[col_name] = df['Timestamp'].diff().fillna(0)

        
        #### TO HERE ####

        df = df.sort_values(by=['temp_idx'], axis=0)
        directory = os.path.join(exp_dir,col_name)
        os.makedirs(directory, exist_ok=True)
        df[col_name].to_csv(os.path.join(directory,f"{mode}.csv"), index=False)

    with open(os.path.join(directory,"feature_desc.json"),"w") as f:
        data_dict["feature_name"] = col_name
        json.dump(data_dict,f)

# data_dict = {
#     "train": %Original_Train_dataset_directory_Here%,
#     "val": %Original_Val_dataset_directory_Here%,
#     "test": %Original_Test_dataset_directory_Here%,
# }

data_dict = {
    "train": '/opt/ml/input/data/train_dataset/cv_train_cached.csv',
    "val": '/opt/ml/input/data/train_dataset/cv_val_cached.csv',
    "test": '/opt/ml/input/data/train_dataset/test_data.csv',
}
feature_creator('/opt/ml/input/data/features/', "elapsed_time", data_dict)

In [ ]:
data_dict = {
    "train": '/opt/ml/input/data/train_dataset/cv_train_cached.csv',
    "val": '/opt/ml/input/data/train_dataset/cv_val_cached.csv',
    "test": '/opt/ml/input/data/train_dataset/test_data.csv',
}
feature_creator('/opt/ml/input/data/features/', "elapsed_time", data_dict)

In [ ]:
df = pd.read_csv('/opt/ml/input/data/train_dataset/test_data.csv', parse_dates=['Timestamp'])

In [ ]:
df['Timestamp']

## TRAIN VS TEST


Train과 Test는 같은 Column을 가지고 있으며, Row의 수는 다음과 같다.
약 8.7:1의 비율이다

In [ ]:
train_df.columns == test_df.columns

In [ ]:
print("train_dataset length: ", len(train_df),"vs", "test_dataset length: ", len(test_df), ", ", len(train_df)/len(test_df))

특히, 우리가 예측해야할 문제의 answerCode column은 0이나 1 대신 -1로 마스킹되어 있으며, 총 744개의 문제를 예측해야한다. 

In [ ]:
test_df[test_df['answerCode'] == -1]

Train과 Test 데이터셋의 unique feature의 수

In [ ]:
print(f"""--- Train_dataset INFORMATIONS ---
userID           : {train_df.userID.nunique()}
assessmentItemID : {train_df.assessmentItemID.nunique()}
testID           : {train_df.testId.nunique()}
mean answer rate : {train_df.answerCode.sum() / train_df.shape[0] * 100:.2f}%
KnowledgeTag     : {train_df.KnowledgeTag.nunique()}
{'-'*26}""")

In [ ]:
print(f"""--- Test_dataset INFORMATIONS ---
userID           : {test_df.userID.nunique()}
assessmentItemID : {test_df.assessmentItemID.nunique()}
testID           : {test_df.testId.nunique()}
mean answer rate : {test_df[test_df['answerCode'] != -1]['answerCode'].sum() / test_df.shape[0] * 100:.2f}%
KnowledgeTag     : {test_df.KnowledgeTag.nunique()}
{'-'*26}""")

In [ ]:
train_unique_userID = pd.unique(train_df.userID)
train_unique_assessmentItemID = pd.unique(train_df.assessmentItemID)
train_unique_testId = pd.unique(train_df.testId)
train_unique_KnowledgeTag = pd.unique(train_df.KnowledgeTag)

각 column의 value의 경우, userID는 하나도 일치하지 않고 나머지 comlumn 모두 일치합니다.

In [ ]:

print(f"""--- Test_dataset INFORMATIONS ---
train에는 없는 userID           : {len([i for i in tqdm(pd.unique(test_df.userID)) if i not in train_unique_userID])}
train에는 없는 assessmentItemID : {len([i for i in tqdm(pd.unique(test_df.assessmentItemID)) if i not in train_unique_assessmentItemID])}
train에는 없는 testID           : {len([i for i in tqdm(pd.unique(test_df.testId)) if i not in train_unique_testId])}
train에는 없는 KnowledgeTag     : {len([i for i in tqdm(pd.unique(test_df.KnowledgeTag)) if i not in train_unique_KnowledgeTag])}
{'-'*26}""")

하지만 train 데이터셋과 test 데이터셋 간의 완전히 겹치는 데이터는 없습니다.
answerCode가 -1인 Row를 빼면 학습 데이터로 활용가능할 것 같습니다.

In [ ]:
df = pd.merge(train_df, test_df, on=list(test_df.columns), how='outer', indicator='Exist')
df['Exist'] = np.where(df.Exist == 'both', True, False)

In [ ]:
df[df['Exist']==True]

In [ ]:
len(list(test_df.groupby('userID')))

In [ ]:
new_df = train_df.sort_values(by=['userID','Timestamp'], axis=0)
columns = ['userID', 'assessmentItemID', 'testId', 'answerCode', 'KnowledgeTag']
group = new_df[columns].groupby('userID').apply(
        lambda r: (
            r['testId'].values, 
            r['assessmentItemID'].values,
            r['KnowledgeTag'].values,
            r['answerCode'].values
        )
    )


In [ ]:
group

In [ ]:
test_df[test_df['answerCode'] == -1]

In [ ]:
new_column = train_df['assessmentItemID'].apply(lambda x : x[:3])
new_column.name = "grade"

In [ ]:
new_column

In [ ]:
new_train_df = train_df.head(40)

In [ ]:
train_df['answerCode'].mean()

In [ ]:
avgs = train_df.groupby('userID')['answerCode'].mean()

In [ ]:
avgs

In [ ]:
train_df.loc[train_df['userID'],'avg'] = 

In [ ]:
train_df.sample(20)
    

In [ ]:
new_train_df[lambda x : x['answerCode']==1]

In [ ]:
len(train_df)

In [ ]:
train_df

In [ ]:
temp = train_df.groupby('userID')['answerCode'].transform('mean')

In [ ]:
temp

In [ ]:
user_acc = train_df.groupby('userID')['answerCode'].mean()
        

In [ ]:
user_acc

In [ ]:
train_df['userID'].apply(lambda x: user_acc[x])

In [ ]:
user_acc.index

In [ ]:
train_df